# QA Generation
This notebook shows how to use the `QAGenerationChain` to come up with question-answer pairs over a specific document.
This is important because often times you may not have data to evaluate your question-answer system over, so this is a cheap and lightweight way to generate it!


And You can find the origin notebook in [LangChain example](https://github.com/hwchase17/langchain/blob/master/docs/use_cases/evaluation/qa_generation.ipynb), and this example will show you how to set the LLM with GPTCache so that you can cache the data with LLM.

## Go into GPTCache

Please [install gptcache](https://gptcache.readthedocs.io/en/latest/index.html#) first, then we can initialize the cache.There are two ways to initialize the cache, the first is to use the map cache (exact match cache) and the second is to use the DataBse cache (similar search cache), it is more recommended to use the second one, but you have to install the related requirements.

> We will use [Milvus](https://milvus.io/) vector database to do similarity retrieval, so we need to install and start Milvus.

In [1]:
! pip install -q gptcache langchain milvus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00


In [2]:
from milvus import default_server
default_server.start()

Before running the example, make sure the `OPENAI_API_KEY` environment variable is set by executing `echo $OPENAI_API_KEY`. If it is not already set, it can be set by using `export OPENAI_API_KEY=YOUR_API_KEY` on Unix/Linux/MacOS systems or `set OPENAI_API_KEY=YOUR_API_KEY` on Windows systems. And there is `get_msg_func` for the cache settings:

> We can run `os.environ` to set the environment variable in colab.

In [4]:
import os
openai_api_key = os.environ.get("OPENAI_API_KEY")

# get the content(only question) form the prompt to cache
def get_msg_func(data, **_):
    return data.get("messages")[-1].content

### 1. Init for exact match cache

In [7]:
from gptcache import cache
cache.init(pre_embedding_func=get_msg_func)
cache.set_openai_key()

### 2. Init for similar match cache

When initializing gptcahe, the following four parameters are configured:

- `pre_embedding_func`: pre-processing before extracting feature vectors, it will use the `get_msg_func` method
- `embedding_func`: the method to extract the text feature vector
- `data_manager`: DataManager for cache management
- `similarity_evaluation`: the evaluation method after the cache hit

The `data_manager` is used to audio feature vector, response text in the example, it takes [Milvus](https://milvus.io/docs) (please make sure it is started), you can also configure other vector storage, refer to [VectorBase API](https://gptcache.readthedocs.io/en/latest/references/manager.html#module-gptcache.manager.vector_data).

In [6]:
from gptcache import cache
from gptcache.embedding import Onnx
from gptcache.manager import CacheBase, VectorBase, get_data_manager
from gptcache.similarity_evaluation.distance import SearchDistanceEvaluation


onnx = Onnx()
cache_base = CacheBase('sqlite')
vector_base = VectorBase('milvus', host='127.0.0.1', port='19530', dimension=onnx.dimension)
data_manager = get_data_manager(cache_base, vector_base)
cache.init(
    pre_embedding_func=get_msg_func,
    embedding_func=onnx.to_embeddings,
    data_manager=data_manager,
    similarity_evaluation=SearchDistanceEvaluation(),
    )
cache.set_openai_key()

start to install package: onnxruntime==1.14.1
successfully installed package: onnxruntime==1.14.1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/827 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/46.9M [00:00<?, ?B/s]

start to install package: pymilvus
successfully installed package: pymilvus


ContextualVersionConflict: (grpcio 1.60.1 (/usr/local/lib/python3.10/dist-packages), Requirement.parse('grpcio<=1.60.0,>=1.49.1'), {'pymilvus'})

After initializing the cache, you can use the LangChain Chat Models with `gptcache.adapter.langchain_models`. At this point **gptcache** will cache the answer, the only difference from the original example is to change `chat=ChatOpenAI(temperature=0)` to `chat = LangChainChat(chat=ChatOpenAI(temperature=0))`, which will be commented in the code block.

Then you will find that it will be more fast when search the similar content, let's play with it.

## Getting Started

In [8]:
! wget https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt -O state_of_the_union.txt

--2024-02-21 12:42:23--  https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-02-21 12:42:23 ERROR 404: Not Found.



In [9]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter

text_splitter =  RecursiveCharacterTextSplitter(chunk_overlap=500, chunk_size=2000)

In [15]:
loader = TextLoader("./test.txt")

In [16]:
doc = loader.load()[0]

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import QAGenerationChain
from gptcache.adapter.langchain_models import LangChainChat

# chat = ChatOpenAI(temperature=0) # using the following code to cache with gptcache
chat = LangChainChat(chat=ChatOpenAI(temperature=0))

chain = QAGenerationChain.from_llm(chat, text_splitter=text_splitter)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [18]:
qa = chain.run(doc.page_content)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [32]:
qa[19]

{'question': "What is the instructor's plan for the module in terms of research proposals?",
 'answer': 'The instructor plans to give more details later on in the module to ensure that students come up with really good research proposals.'}